# The IPython widgets, now in IHaskell !!

It is highly recommended that users new to jupyter/ipython take the *User Interface Tour* from the toolbar above (Help -> User Interface Tour).

> This notebook introduces the [IPython widgets](https://github.com/ipython/ipywidgets), as implemented in [IHaskell](https://github.com/gibiansky/IHaskell). The `Button` widget is also demonstrated as a live action example.

### The Widget Hierarchy

These are all the widgets available from IPython/Jupyter.

#### Uncategorized Widgets

+ Button
+ Image*Widget*
+ Output*Widget*

#### Box Widgets

+ Box
+ FlexBox
+ Accordion
+ Tab*Widget*

#### Boolean Widgets

+ CheckBox
+ ToggleButton

#### Integer Widgets

+ IntText
+ BoundedIntText
+ IntProgress
+ IntSlider
+ IntRangeSlider

#### Float Widgets

+ FloatText
+ BoundedFloatText
+ FloatProgress
+ FloatSlider
+ FloatRangeSlider

#### Selection Widgets

+ Selection
+ Dropdown
+ RadioButtons
+ Select
+ SelectMultiple
+ ToggleButtons

#### String Widgets

+ HTML*Widget*
+ Latex*Widget*
+ TextArea
+ Text*Widget*

### Using Widgets

#### Necessary Extensions and Imports

All the widgets and related functions are available from a single module, `IHaskell.Display.Widgets`. It is strongly recommended that users use the `OverloadedStrings` extension, as widgets make extensive use of `Text`.

In [1]:
{-# LANGUAGE OverloadedStrings #-}
import IHaskell.Display.Widgets

The module can be imported unqualified. Widgets with common names, such as `Text`, `Image` etc. have a `-Widget` suffix to prevent name collisions.

#### Widget interface

Each widget has different properties, but the surface level API is the same.

Every widget has:

1. A constructor:
    An `IO <widget>` value/function of the form `mk<widget_name>`.
2. A set of properties, which can be manipulated using `setField` and `getField`.

The `setField` and `getField` functions have nasty type signatures, but they can be used by just intuitively understanding them.

In [2]:
:t setField

setField :: forall f w. (RElemOf f (WidgetFields w), ToKey f, IHaskellWidget (IPythonWidget w), ToPairs (Attr f)) => IPythonWidget w -> FieldType f -> IO ()

The `setField` function takes three arguments, the first of which is a typelevel argument:

1. A type level `Field`
2. A widget
3. A value for the `Field`

In [3]:
:t getField

getField :: forall f w. RElemOf f (WidgetFields w) => IPythonWidget w -> IO (FieldType f)

The `getField` function takes a type-level `Field` and a `Widget` value and returns the value of that `Field` for the `Widget`.

Another utility function is `properties`, which shows all properties of a widget.

In [4]:
:t properties

properties :: forall w. IPythonWidget w -> IO ()

#### Displaying Widgets

IHaskell automatically displays anything *displayable* given to it directly.

In [5]:
-- Showables
1 + 2
"abc"

3

"abc"

Widgets can either be displayed this way, or explicitly using the `display` function from `IHaskell.Display`.

In [6]:
import IHaskell.Display
:t display

display :: forall a. IHaskellDisplay a => a -> IO Display

#### Multiple displays

A widget can be displayed multiple times. All these *views* are representations of a single object, and thus are linked.

When a widget is created, a model representing it is created in the frontend. This model is used by all the views, and any modification to it propagates to all of them.

#### Closing widgets

Widgets can be closed using the `closeWidget` function.

In [7]:
:t closeWidget

closeWidget :: forall w. IHaskellWidget w => w -> IO ()

### Our first widget: `Button`

Let's play with buttons as a starting example:

As noted before, all widgets have a constructor of the form `mk<Widget>`. Thus, to create a `Button`, we use `mkButton`.

In [8]:
button <- mkButton     -- Construct a Button
:t button

button :: Button

Widgets can be displayed by just entering them into a cell.

In [9]:
button                 -- Display the button

To view a widget's properties, we use the `properties` function. It also shows the type represented by the `Field`, which generally are not visible in type signatures due to high levels of type-hackery.

In [10]:
-- The button widget has many properties.
properties button

ViewModule ::: Text
ViewModuleVersion ::: Text
ModelModule ::: Text
ModelModuleVersion ::: Text
ModelName ::: Text
ViewName ::: Text
DOMClasses ::: [Text]
Tabbable ::: Maybe Bool
Tooltip ::: Maybe Text
Layout ::: IPythonWidget LayoutType
DisplayHandler ::: IO ()
Description ::: Text
DescriptionAllowHtml ::: Maybe Bool
Style ::: StyleWidget
Disabled ::: Bool
Icon ::: Text
ButtonStyleField ::: ButtonStyleValue
ClickHandler ::: IO ()

Let's try making the button widget wider.

In [12]:
import qualified IHaskell.Display.Widgets.Layout as L
btnLayout <- getField @Layout button
setField @L.Width btnLayout $ Just "100%"

There is a lot that can be customized. For example:

In [14]:
setField @Description button "Click Me (._.\")"
setField @ButtonStyleField button SuccessButton
setField @L.BorderTop btnLayout $ Just "ridge 2px"
setField @L.Padding btnLayout $ Just "10"
setField @L.Height btnLayout $ Just "7em"

The button widget also provides a click handler. We can make it do anything, except console input. Universally, no widget event can trigger console input.

In [16]:
setField @ClickHandler button $ putStrLn "fO_o"
button -- Displaying again for convenience

Now try clicking the button, and see the output.

> Note: If you display to stdout using Jupyter Lab, it will be displayed in a log entry, not as the cell output.

We can't do console input, though, but you can always use another widget! See the other notebooks with examples for more information

In [17]:
setField @ClickHandler button $ getLine >>= putStrLn